In [70]:
import glob
import os
import re
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# import lightgbm as lgb

warnings.filterwarnings('ignore')

In [71]:
train = pd.read_csv('../processed_data/train_v7.csv')
test = pd.read_csv('../processed_data/test_v7.csv')

In [72]:
train['location']

0              東京都北区滝野川３丁目
1              東京都中央区月島３丁目
2              東京都渋谷区笹塚２丁目
3        東京都杉並区高円寺南２丁目23-2
4           東京都葛飾区金町３丁目7-2
               ...        
31462          東京都板橋区蓮根２丁目
31463    東京都世田谷区太子堂５丁目17-1
31464          東京都江東区南砂４丁目
31465          東京都中野区中野２丁目
31466          東京都豊島区千川２丁目
Name: location, Length: 31467, dtype: object

In [73]:
train['location'].str.extract('(.+)都(.*区)(.*)',expand=True)

,0,1,2
0,東京,北区,滝野川３丁目
1,東京,中央区,月島３丁目
2,東京,渋谷区,笹塚２丁目
3,東京,杉並区,高円寺南２丁目23-2
4,東京,葛飾区,金町３丁目7-2
...,...,...,...
31462,東京,板橋区,蓮根２丁目
31463,東京,世田谷区,太子堂５丁目17-1
31464,東京,江東区,南砂４丁目
31465,東京,中野区,中野２丁目


In [74]:
layout = pd.concat([train['layout'],test['layout']])

In [75]:
layout.value_counts()

1K            23661
1R            10177
1LDK           9702
2LDK           5130
1DK            3781
2DK            3715
3LDK           2240
2K             1807
3DK             851
1LDK+S(納戸)      487
2LDK+S(納戸)      326
4LDK            250
3LDK+S(納戸)      133
1K+S(納戸)        130
3K              102
5LDK             38
2DK+S(納戸)        37
4DK              33
1DK+S(納戸)        27
4LDK+S(納戸)       24
4K               12
3DK+S(納戸)        10
6LDK              9
5LDK+S(納戸)        9
5DK               8
5K                7
6LDK+S(納戸)        4
2K+S(納戸)          3
5DK+S(納戸)         3
4DK+S(納戸)         2
6DK               2
1LK+S(納戸)         2
3K+S(納戸)          2
8LDK              1
2R                1
6K                1
11R               1
1LK               1
Name: layout, dtype: int64

In [76]:
test[test['layout']=='11R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
8483,39954,東京都世田谷区桜上水５丁目,京王線\t桜上水駅\t徒歩5分\t\t京王線\t上北沢駅\t徒歩10分\t\t東急世田谷線\...,11R,64年0ヶ月,南,16m2,1,専用バス／\t専用トイレ／\tシャワー,ガスコンロ／\t冷蔵庫あり,...,5971,262,1,12112,12311,1,47969,17695,54,45052


In [77]:
test[test['layout']=='2R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
24546,56017,東京都北区志茂２丁目,京浜東北線\t赤羽駅\t徒歩10分\t\t南北線\t志茂駅\t徒歩4分\t\t南北線\t赤羽...,2R,55年10ヶ月,南,14.8m2,2,共同トイレ,NaN,...,2061,8,7,17201,12311,1,11019,17695,1,45052


In [78]:
train.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contact_period', 'log_y',
       'area_num', 'age_month', 'max_floor', 'room_num', 'L', 'D', 'K', 'R',
       'S', 'north', 'structure_orderd', 'walk_time', 'min_time', 'avg_time',
       'bicycle_parking', 'car_parking', 'bike_parking', 'toilet', 'bath',
       'sm_doku', 'kanso', 'onsui', 'oidaki', 'b_t_split', '23ku',
       '23ku_mean_std', '23ku_countall', 'area_num_countall', 'age_countall',
       'floor_countall', 'max_floor_countall', 'layout_countall',
       'room_num_countall', 'direction_countall', 'facilities_countall',
       'contact_period_countall'],
      dtype='object')

In [79]:
c_period = pd.concat([train['contact_period'],test['contact_period']])

In [80]:
c_period.value_counts()

2年間                              45052
2年間\t※この物件は\t定期借家\tです。            1888
5年間\t※この物件は\t定期借家\tです。             506
3年間\t※この物件は\t定期借家\tです。             443
1年間                                163
                                 ...  
12年間                                 1
2025年8月まで\t※この物件は\t定期借家\tです。         1
2021年1月まで\t※この物件は\t定期借家\tです。         1
3年9ヶ月間\t※この物件は\t定期借家\tです。            1
2020年11月まで\t※この物件は\t定期借家\tです。        1
Name: contact_period, Length: 79, dtype: int64

In [81]:
def teiki_syakuya(c_period):
    res = []
    for c in c_period.fillna(''):
        if '定期借家' in c:
            res.append(1)
        else:
            res.append(0)
    return res

In [82]:
c_period.isnull().sum()

14098

In [103]:
train_v7 = pd.read_csv('../processed_data/train_v7.csv')
# train_v7.isnull().sum()

In [105]:
train_v7.isnull().sum()

id                            0
y                             0
location                      0
access                        0
layout                        0
age                           0
direction                  2749
area                          0
floor                         0
bath_toilet                 329
kitchen                     621
broadcast_com              3440
facilities                   82
parking                    5285
enviroment                 9430
structure                     0
contact_period             7117
log_y                         0
area_num                      0
age_month                     0
max_floor                     0
room_num                      0
L                             0
D                             0
K                             0
R                             0
S                             0
north                         0
structure_orderd              0
walk_time                     0
min_time                      0
avg_time

In [102]:
train_v7['parking'].value_counts()

駐車場\t無                                           3269
駐輪場\t空有\t駐車場\t無\tバイク置き場\t無                       2374
駐車場\t無\t駐輪場\t無\tバイク置き場\t無                        1832
駐輪場\t空有                                          1246
駐輪場\t空有\tバイク置き場\t無                                957
                                                 ... 
駐車場\t空有\t23,000円\t(※空き要確認)                          1
駐車場\t空無\t36,000円(税込)                                1
駐車場\t空有\t32,400円\t駐輪場\t空有\t0円                       1
駐車場\t空有(6台)\t32,400円                                1
駐車場\t空有\t14,000円\t駐輪場\t空有\t0円\tバイク置き場\t空有\t0円       1
Name: parking, Length: 3911, dtype: int64

In [106]:
train_v7['bath_toilet'].value_counts()

専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立／\t脱衣所    4606
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立／\t脱衣所           2936
専用バス／\t専用トイレ／\tシャワー                                                      2133
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー                                            1495
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立           873
                                                                         ... 
バスなし                                                                        1
専用バス／\tシャワー／\t追焚機能                                                          1
専用バス／\t専用トイレ／\tシャワー／\t温水洗浄便座／\t脱衣所                                          1
共同バス／\t共同トイレ／\tバス・トイレ別／\tシャワー                                               1
専用バス／\t専用トイレ／\tシャワー／\t追焚機能／\t温水洗浄便座／\t脱衣所                                   1
Name: bath_toilet, Length: 360, dtype: int64

In [107]:
kitchen = pd.concat([train['kitchen'],test['kitchen']])
kitchen

0              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
1                      ガスコンロ／\tシステムキッチン\t／\t給湯
2              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
3                                   ガスコンロ／\t給湯
4                                  給湯／\t独立キッチン
                         ...                  
31257          ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
31258      ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン
31259                       ガスコンロ／\tコンロ3口／\t給湯
31260    IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン
31261                  コンロ2口／\tシステムキッチン\t／\t給湯
Name: kitchen, Length: 62729, dtype: object

In [108]:
kitchen.unique()

array(['ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯', 'ガスコンロ／\tシステムキッチン\t／\t給湯',
       'ガスコンロ／\t給湯', '給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ2口／\tシステムキッチン', 'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯',
       'ガスコンロ／\tコンロ2口／\t給湯', nan, 'コンロ2口／\t給湯', '給湯',
       'IHコンロ\t／\tコンロ3口／\tシステムキッチン',
       'IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン', 'ガスコンロ／\tコンロ1口／\t給湯',
       'ガスコンロ／\tコンロ1口／\t給湯／\t冷蔵庫あり',
       'ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯／\t独立キッチン',
       'コンロ3口／\tシステムキッチン\t／\t給湯', 'IHコンロ\t／\tシステムキッチン\t／\t給湯',
       'コンロ設置可（コンロ2口）',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\tカウンターキッチン\t／\t給湯／\tL字キッチン',
       'ガスコンロ／\tコンロ1口／\t給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\tカウンターキッチン\t／\t給湯',
       'ガスコンロ／\tコンロ設置可（口数不明）／\t給湯',
       'ガスコンロ／\tコンロ2口／\t給湯／\t独立キッチン／\tL字キッチン', 'システムキッチン',
       'ガスコンロ／\tコンロ2口', '電気コンロ／\t給湯',
       'IHコンロ\t／\tコンロ2口／\tシステムキッチン\t／\t給湯／\tL字キッチン',
       'コンロ2口／\tシステムキッチン\t／\t給湯', 'コンロ2口／\tシステムキッチン\t／\t独立キッチン',
       'ガスコンロ／\tコンロ2口／\tシステム

In [110]:
train[train['kitchen'].isnull()].loc[:,['kitchen','layout']]

,kitchen,layout
11,NaN,1K
73,NaN,1LDK
74,NaN,1R
134,NaN,1K
139,NaN,1K
...,...,...
31008,NaN,1R
31027,NaN,1R
31075,NaN,1R
31155,NaN,2K


In [112]:
hoge = []
for k in kitchen.fillna(''):
    split_k = k.split('\t')
    hoge+=split_k

In [113]:
set(hoge)

{'',
 'IHコンロ',
 'L字キッチン',
 'カウンターキッチン',
 'ガスコンロ',
 'ガスコンロ／',
 'コンロ1口',
 'コンロ1口／',
 'コンロ2口',
 'コンロ2口／',
 'コンロ3口',
 'コンロ3口／',
 'コンロ4口以上',
 'コンロ4口以上／',
 'コンロ設置可（コンロ1口）',
 'コンロ設置可（コンロ1口）／',
 'コンロ設置可（コンロ2口）',
 'コンロ設置可（コンロ2口）／',
 'コンロ設置可（コンロ3口）',
 'コンロ設置可（コンロ3口）／',
 'コンロ設置可（コンロ4口以上）／',
 'コンロ設置可（口数不明）',
 'コンロ設置可（口数不明）／',
 'システムキッチン',
 '冷蔵庫あり',
 '冷蔵庫あり／',
 '独立キッチン',
 '独立キッチン／',
 '給湯',
 '給湯／',
 '電気コンロ',
 '電気コンロ／',
 '／'}

In [114]:
train.groupby('kitchen').describe()['y']

,count,mean,std,min,25%,50%,75%,max
kitchen,,,,,,,,
IHコンロ,110.0,67331.818182,24792.651811,35000.0,53000.0,62000.0,70000.0,175000.0
IHコンロ\t／\tカウンターキッチン\t／\t給湯,10.0,124750.000000,67242.203183,62000.0,73750.0,113750.0,147000.0,280000.0
IHコンロ\t／\tコンロ1口,79.0,71602.531646,34557.124668,28000.0,58000.0,65000.0,78400.0,334800.0
IHコンロ\t／\tコンロ1口／\tカウンターキッチン,1.0,64000.000000,NaN,64000.0,64000.0,64000.0,64000.0,64000.0
IHコンロ\t／\tコンロ1口／\tカウンターキッチン\t／\t給湯,5.0,102200.000000,41166.734143,76000.0,79000.0,79000.0,104000.0,173000.0
...,...,...,...,...,...,...,...,...
電気コンロ／\tシステムキッチン\t／\t給湯／\t冷蔵庫あり,3.0,64000.000000,1732.050808,63000.0,63000.0,63000.0,64500.0,66000.0
電気コンロ／\tシステムキッチン\t／\t給湯／\t独立キッチン,1.0,76000.000000,NaN,76000.0,76000.0,76000.0,76000.0,76000.0
電気コンロ／\t給湯,61.0,62947.540984,10804.838524,41000.0,55000.0,61000.0,70000.0,93000.0


In [115]:
train.groupby('facilities').describe()['y']

,count,mean,std,min,25%,50%,75%,max
facilities,,,,,,,,
24時間換気システム\t／\t室内洗濯機置場／\tタイル張り\t公営水道／\t下水／\t都市ガス,2.0,64500.000000,707.106781,64000.0,64250.0,64500.0,64750.0,65000.0
2面採光\t公営水道／\t下水／\tプロパンガス,1.0,62000.000000,NaN,62000.0,62000.0,62000.0,62000.0,62000.0
2面採光\t公営水道／\t下水／\t都市ガス,5.0,53040.000000,9513.569257,46000.0,46000.0,48200.0,57000.0,68000.0
2面採光\t都市ガス,1.0,80000.000000,NaN,80000.0,80000.0,80000.0,80000.0,80000.0
2面採光／\tクッションフロア／\t室内洗濯機置場\t公営水道／\t下水／\t都市ガス,1.0,48000.000000,NaN,48000.0,48000.0,48000.0,48000.0,48000.0
...,...,...,...,...,...,...,...,...
水道その他／\t下水／\t都市ガス,3.0,38733.333333,3002.221400,37000.0,37000.0,37000.0,39600.0,42200.0
洗濯機置場なし\t公営水道／\t下水／\t都市ガス,2.0,46000.000000,8485.281374,40000.0,43000.0,46000.0,49000.0,52000.0
洗濯機置場なし\t公営水道／\t都市ガス,1.0,40000.000000,NaN,40000.0,40000.0,40000.0,40000.0,40000.0


In [116]:
train.groupby('broadcast_com').describe()['y']

,count,mean,std,min,25%,50%,75%,max
broadcast_com,,,,,,,,
BSアンテナ,315.0,107991.587302,40671.573752,29500.0,82750.0,101000.0,125250.0,350000.0
BSアンテナ／\tインターネット使用料無料,27.0,100814.814815,15417.989086,65000.0,92250.0,100500.0,111000.0,143500.0
CATV,979.0,100575.464760,80371.396048,33000.0,66000.0,82100.0,110000.0,1660000.0
CATV／\tBSアンテナ,216.0,122553.175926,92142.335408,48000.0,79000.0,98000.0,133500.0,730000.0
CATV／\tBSアンテナ／\tインターネット使用料無料,6.0,85250.000000,27777.238884,55000.0,70875.0,73000.0,107000.0,122000.0
...,...,...,...,...,...,...,...,...
高速インターネット／\t光ファイバー／\tCATV,88.0,109873.863636,79254.167727,45000.0,71000.0,90750.0,120000.0,598000.0
高速インターネット／\t光ファイバー／\tCATV／\tBSアンテナ,25.0,98160.000000,45456.646012,53000.0,78500.0,80000.0,105000.0,256000.0
高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ,6.0,249500.000000,106893.872603,45000.0,245000.0,276000.0,313000.0,340000.0
